# Example - Using Python to connect to PG Apache AGE

- reference: https://age.apache.org/getstarted/quickstart
- to install `apache-age-python`, code compile is currently required:

```shell
sudo apt-get -y install gcc libpq-dev "postgresql-client-${VER_PG}"
```

In [ ]:
!  sudo apt-get update && sudo apt-get install -y lsb-release gcc \
&& echo "deb https://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" | sudo tee /etc/apt/sources.list.d/pgdg.list \
&& curl "https://www.postgresql.org/media/keys/ACCC4CF8.asc" | sudo tee /etc/apt/trusted.gpg.d/postgresql.asc \
&& sudo apt-get update && sudo apt-get -y install libpq-dev postgresql-client-16 \
&& pip install apache-age-python pandas

# Connect to Postgresql

In [ ]:
import age
import psycopg2
import pandas as pd

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 999

In [ ]:
conn = psycopg2.connect(host="db-postgres", port="5432", dbname="postgres", user="postgres", password="postgres")
conn.set_session(autocommit=True)

In [ ]:
def Q(sql: str):
    return pd.read_sql(sql, conn, )

## Check if PG extension `age` is installed

In [ ]:
Q('''SELECT a.name, e.extversion AS installed, a.default_version AS avaliable, a.comment --e.extowner, e.extnamespace, e.extrelocatable
    FROM pg_available_extensions AS a LEFT JOIN pg_extension AS e ON a.name = e.extname
    ORDER BY name;
''')

# Start to Execute Cypher query with the help of `age`

In [ ]:
with conn.cursor() as cur:
    cur.execute("""
LOAD 'age';
SET search_path = ag_catalog, "$user", public;
""")

## Create a graph called `graph_a`

In [ ]:
Q('''SELECT * FROM ag_catalog.create_graph('graph_a');''')

## List all graphs using SQL table `ag_catalog.ag_graph`

In [ ]:
Q('''SELECT * FROM ag_catalog.ag_graph;''')

## List all all label information using SQL table `ag_catalog.ag_label`

In [ ]:
Q('''
SELECT * FROM ag_catalog.ag_label;
''')

## Create an entity Type `Person` in graph `graph_a`

In [ ]:
Q('''
SELECT ag_catalog.create_vlabel('graph_a', 'Person');
''')

## Create two nodes and an edge

In [ ]:
Q('''
SELECT * FROM cypher('graph_a', $$
CREATE (:Person {name: 'Daedalus'})-[:FATHER_OF]->(:Person {name: 'Icarus'})
$$) AS (a agtype);
''')

## List all information in `ag_catalog.ag_label`

In [ ]:
Q('''
SELECT * FROM ag_catalog.ag_label ORDER BY kind, name;
''')

## Execute a Cypher query

In [ ]:
Q('''
SELECT * FROM cypher('graph_a', $$ 
    MATCH (v:Person)
    RETURN v.name
$$) AS (result1 agtype);
''')